In [1]:
#@title import
%%capture
!pip install gnews newspaper3k
from gnews import GNews
from tqdm.notebook import tqdm_notebook as tqdm
from datetime import datetime, timedelta
import spacy
import re
import time
import pandas as pd
from google.colab import files

+ function to return news (dict)

In [ ]:
def get_related_news(google_news, keyword:str, start_date:datetime, end_date:datetime, bar=None)->list[dict]:
  google_news.start_date = (start_date.year, start_date.month, start_date.day)
  google_news.end_date = (end_date.year, end_date.month, end_date.day)

  if (bar is None):
    bar = tqdm(total=(end_date-start_date).days+1, desc="Getting News")
  # Get the news results
  results = google_news.get_news(keyword)
  num = len(results)

  if ((num >=99) and ((end_date - start_date)> timedelta(days=4))):
      # Recursively call the function for 1/2 the time and add them up
      mid_date = start_date + (end_date - start_date) / 2
      mid_date = datetime(mid_date.year, mid_date.month, mid_date.day)

      # Merge the results
      results = get_related_news(google_news, keyword, mid_date+timedelta(days=1), end_date, bar)\
              + get_related_news(google_news, keyword, start_date, mid_date, bar)

      #Check tqdm bar and close it
      if (bar.total == bar.n):
        bar.close()

      # Return direclty since results are already sorted
      return results

  sorted_results= sorted(results,
                         key=lambda x: datetime.strptime(x['published date'], "%a, %d %b %Y %H:%M:%S %Z"),
                         reverse=True)
  # Update tqdm bar
  update_proportion = (end_date - start_date).days+1
  bar.update(update_proportion)

  if (bar.total == bar.n):
    bar.close()

  time.sleep(10)

  return sorted_results

+ helper function for saving to csv, full dict and random 100

In [20]:
def save_and_download_csv(df, keyword):
    df.to_csv(f"{keyword}.csv", index=False)
    files.download(f"{keyword}.csv")

In [30]:
def save_500_sample(df,keyword):
    df_500 = df.sample(500)
    save_and_download_csv(df_500, f"{keyword}_500")

+ scraaaaping

In [32]:
# keyword = "COVID"
keyword = "symptom"

In [ ]:
start_date, end_date = datetime(2019, 10, 1 , 0, 0, 0), datetime(2023,10,1,0,0,0)

google_news = GNews()

results = get_related_news(google_news, keyword, start_date, end_date)
print(f"Num of news: {len(results)}")

Getting News:   0%|          | 0/1462 [00:00<?, ?it/s]

Num of news: 27104


In [33]:
# df = pd.DataFrame(results)
df = pd.read_csv(f"{keyword}.csv")

In [34]:
# save_and_download_csv(df, keyword)
save_500_sample(df, keyword)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
covid = pd.read_csv(f"{keyword}.csv")

In [ ]:
covid.head()

,title,description,published date,url,publisher
0,Do I need any more COVID vaccinations? - Harva...,Do I need any more COVID vaccinations? Harvar...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiXGh0d...,"{'href': 'https://www.health.harvard.edu', 'ti..."
1,COVID-19 report details continuing Veteran hea...,COVID-19 report details continuing Veteran hea...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiR2h0d...,"{'href': 'https://news.va.gov', 'title': 'VA.g..."
2,What's behind the messy rollout of this fall's...,What's behind the messy rollout of this fall's...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiYWh0d...,"{'href': 'https://www.pbs.org', 'title': 'PBS ..."
3,New study reveals alarming cardiovascular risk...,New study reveals alarming cardiovascular risk...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,"{'href': 'https://www.news-medical.net', 'titl..."
4,What parents need to know about common viruses...,What parents need to know about common viruses...,"Sun, 01 Oct 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiZ2h0d...,"{'href': 'https://www.uchicagomedicine.org', '..."


In [49]:
keyword = "COVID"
# keyword = "symptom"

In [50]:
to_process = pd.read_csv(f"{keyword}_500.csv")

In [51]:
len(to_process)

500

In [52]:
to_process.head()

,title,description,published date,url,publisher
0,California COVID cases are increasing. What yo...,California COVID cases are increasing. What yo...,"Fri, 01 Sep 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiUmh0d...,"{'href': 'https://calmatters.org', 'title': 'C..."
1,U.S. Supreme Court's Sotomayor keeps New York ...,U.S. Supreme Court's Sotomayor keeps New York ...,"Mon, 29 Aug 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.reuters.com', 'title': '..."
2,"Josh Shapiro, Dem. candidate for Pa. gov., tes...","Josh Shapiro, Dem. candidate for Pa. gov., tes...","Tue, 17 May 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVWh0d...,"{'href': 'https://whyy.org', 'title': 'WHYY'}"
3,COVID was the leading cause of law enforcement...,COVID was the leading cause of law enforcement...,"Wed, 11 Jan 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiY2h0d...,"{'href': 'https://abcnews.go.com', 'title': 'A..."
4,Racial disparities in Michigan COVID-19 deaths...,Racial disparities in Michigan COVID-19 deaths...,"Mon, 27 Feb 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.bridgemi.com', 'title': ..."


In [53]:
google_news = GNews()

json_resp = to_process.to_dict(orient='records')
all_texts = []
for item in json_resp:
    time.sleep(1.5)
    article = google_news.get_full_article(item['url'])
    try:
        text = article.text
    except AttributeError:
        text = ""  # Append an empty string if article is None
    all_texts.append(text)

An error occurred while fetching the article: Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/world/us/us-supreme-courts-sotomayor-keeps-new-york-city-covid-vaccine-mandate-2022-08-29/ on URL https://news.google.com/rss/articles/CBMicmh0dHBzOi8vd3d3LnJldXRlcnMuY29tL3dvcmxkL3VzL3VzLXN1cHJlbWUtY291cnRzLXNvdG9tYXlvci1rZWVwcy1uZXcteW9yay1jaXR5LWNvdmlkLXZhY2NpbmUtbWFuZGF0ZS0yMDIyLTA4LTI5L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en
An error occurred while fetching the article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.cedars-sinai.org/newsroom/healthcom-why-even-a-faint-line-on-your-rapid-test-still-means-youre-covid--positive/ on URL https://news.google.com/rss/articles/CBMie2h0dHBzOi8vd3d3LmNlZGFycy1zaW5haS5vcmcvbmV3c3Jvb20vaGVhbHRoY29tLXdoeS1ldmVuLWEtZmFpbnQtbGluZS1vbi15b3VyLXJhcGlkLXRlc3Qtc3RpbGwtbWVhbnMteW91cmUtY292aWQtLXBvc2l0aXZlL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en
An error occurred while fetching the

In [54]:
print(len(all_texts))

500


In [55]:
df = pd.DataFrame(json_resp)
df_text = pd.DataFrame({'texts': all_texts})
df = pd.concat([df, df_text], axis=1)
df.head()

,title,description,published date,url,publisher,texts
0,California COVID cases are increasing. What yo...,California COVID cases are increasing. What yo...,"Fri, 01 Sep 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiUmh0d...,"{'href': 'https://calmatters.org', 'title': 'C...",In summary It’s a confusing time in the COVID ...
1,U.S. Supreme Court's Sotomayor keeps New York ...,U.S. Supreme Court's Sotomayor keeps New York ...,"Mon, 29 Aug 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.reuters.com', 'title': '...",
2,"Josh Shapiro, Dem. candidate for Pa. gov., tes...","Josh Shapiro, Dem. candidate for Pa. gov., tes...","Tue, 17 May 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVWh0d...,"{'href': 'https://whyy.org', 'title': 'WHYY'}","Josh Shapiro, the lone Democratic candidate fo..."
3,COVID was the leading cause of law enforcement...,COVID was the leading cause of law enforcement...,"Wed, 11 Jan 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiY2h0d...,"{'href': 'https://abcnews.go.com', 'title': 'A...",COVID was the leading cause of law enforcement...
4,Racial disparities in Michigan COVID-19 deaths...,Racial disparities in Michigan COVID-19 deaths...,"Mon, 27 Feb 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.bridgemi.com', 'title': ...","In April 2020, shortly after COVID-19 first hi..."


In [56]:
df = df[["published date","texts"]]

In [57]:
df["texts"][99]

'A lesser known but equally crucial dimension in managing and responding to an epidemic is social sciences in health, which can shed light on populations\' understanding, perception and acceptance of the risks associated with diseases. The COVID-19 pandemic is no exception. As the disease spread across the planet, clear differences in perception by populations were observed in different countries, creating fertile ground for investigation for researchers specializing in social sciences applied to health.\n\nSocial sciences are a series of academic disciplines (anthropology, sociology, communication, economics, etc.) whose common thread is the study of human societies and social interactions between individuals, groups and their environments.\n\nDuring the Ebola epidemic in 2015, healthcare procedures and safe burial management went against certain cultural and religious practices, creating very high levels of mistrust among populations regarding healthcare interventions and those respo

+ split sentence tokenization

In [58]:
# Load the English language model
nlp = spacy.load("en_core_web_sm")

def split_sentences(text):
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

sentences = []
for text in df['texts']:
    if text.strip():  # Check if the text is not empty
        sentences.extend(split_sentences(text))

print(sentences[:5])

['In summary It’s a confusing time in the COVID pandemic.', 'A new booster is on the way, but cases are rising and you might want more protection now.', 'Another COVID-19 wave is hitting California as the summer ends and kids head back to school.', 'It’s a familiar story by now, but one that has become perhaps more confusing with time because of changing public health recommendations, new vaccine boosters and our evolving understanding of the virus.', 'There’s no reason to panic, State Epidemiologist Dr. Erica Pan said, with population immunity at high levels.']


+ Write in text file

In [59]:
with open(f'/content/textLine_{keyword}.txt', 'w') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

In [60]:
def remove_empty_lines(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Filter out empty lines
    filtered_lines = [line for line in lines if line.strip()]

    # Write the modified lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(filtered_lines)

In [61]:
remove_empty_lines(f"textLine_{keyword}.txt")

+ Number of tokens

In [62]:
def count_words(filename):
    try:
        with open(filename, 'r') as file:
            content = file.read()
            words = content.split()
            return len(words)
    except FileNotFoundError:
        print("File not found.")
        return 0

filename = f'textLine_{keyword}.txt'
word_count = count_words(filename)
print(f"The file '{filename}' contains {word_count} words.")

The file 'textLine_COVID.txt' contains 391773 words.


In [67]:
from google.colab import files

# filename =

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>